<a href="https://colab.research.google.com/github/GregoryCarver/NumericalAnalysis/blob/main/AnalysisMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerical Analysis Methods

In [ ]:
import autograd.numpy as np
from autograd import grad, jacobian
from numpy import linalg as la

## Iterative Methods

##### __Newton Method__ for systems of nonlinear equations:

In [ ]:
def NewtonSys(currSystem, initialEstimate, tolerance, maxIterations):
  currError = tolerance + 1
  iterationCount = 0
  myJac = jacobian(currSystem)
  currEstimate = initialEstimate

  while currError >= tolerance and iterationCount < maxIterations:
    iterationCount = iterationCount + 1
    delta = -(la.inv(myJac(currEstimate)).dot(currSystem(currEstimate)))
    currEstimate = currEstimate + delta
    currError = la.norm(delta)
  
  if iterationCount >= maxIterations and currError > tolerance:
    print("Reached max iteration count without converging.")
    return

  return currEstimate

##### __Broyden Method__ for systems of nonlinear equations:

In [ ]:
def BroydenSys(currSystem, initialEstimate, tolerance, maxIterations):
  currError = tolerance + 1
  iterationCount = 0
  currBroydenMatrix = np.array([[1.0, 0.0], [0.0, 1.0]])
  #Used for testing
  #currBroydenMatrix = jacobian(currSystem)(initialEstimate)
  currEstimate = initialEstimate

  while currError >= tolerance and iterationCount < maxIterations:
    iterationCount = iterationCount + 1
    oldSystemSolution = currSystem(currEstimate)
    delta = -(la.inv(currBroydenMatrix).dot(oldSystemSolution))
    #print("delta: {}".format(delta))
    currEstimate = currEstimate + delta
    #print("currEstimate: {}".format(currEstimate))
    #print("currFunctSol: {}".format(np.outer(currSystem(currEstimate), delta)))
    #functDiff = currSystem(currEstimate) - oldSystemSolution
    currBroydenMatrix = currBroydenMatrix + (np.outer(currSystem(currEstimate), delta) / delta.dot(delta))
    #print("currBroydenMatrix: {}".format(currBroydenMatrix))
    currError = la.norm(delta)

  if iterationCount >= maxIterations and currError > tolerance:
    print("Reached max iteration count without converging.")
    return

  return currEstimate

##### __Aitken Method__ for root finding(may need to revise, not tested):

In [ ]:
def Aitken(currFunction, initialEstimate, tolerance, maxIterations):
  currError = tolerance + 1
  iterationCount = 0
  currEstimate = initialEstimate

  while currError >= tolerance and iterationCount < maxIterations:
    iterationCount = iterationCount + 1
    solution = currFunction(currEstimate)
    secSolution = currFunction(solution)
    newEstimate = (((currEstimate * secSolution) - solution**2) / (secSolution - (2 * solution) + currEstimate))
    print(newEstimate)
    currError = abs(currEstimate - newEstimate)
    currEstimate = newEstimate
  
  if iterationCount >= maxIterations and currError > tolerance:
    print("Reached max iteration count without converging.")
    return

  return currEstimate

##### __Horner Method__ for polynomial evaluation:

In [ ]:
#Takes a vector of coefficients and a point to evaluate the polynomial at. Returns the result and a vector of new coefficients
def Horner(coefficients, givenPoint):
  #Degree of the polynomial
  degree = coefficients.size
  newCoefficients = np.zeros(degree)
  newCoefficients[0] = coefficients[0]
  for j in range(1, degree):
    newCoefficients[j] = coefficients[j] + (newCoefficients[j - 1] * givenPoint)
  return [newCoefficients[degree - 1], newCoefficients[0:degree]]

##### __Newton-Horner Method__ for root finding(need to fix):

In [ ]:
######################################################Returning with extra zero
def NewtonHorner(coefficients, initialEstimate, tolerance, maxIterations):
  degree = coefficients.size
  roots = np.zeros(degree)
  iterationCounts = np.zeros(degree)
  currCoefficients = coefficients
  
  for i in range(0, degree - 1):
    currError = tolerance + 1
    iterationCount = 0
    currEstimate = initialEstimate

    while currError >= tolerance and iterationCount < maxIterations:
      iterationCount = iterationCount + 1
      resultOne, newCoefficients = Horner(currCoefficients, currEstimate)
      resultTwo, newCoefficients = Horner(newCoefficients, currEstimate)
      newEstimate = currEstimate - (resultOne / resultTwo)
      currError = np.abs(newEstimate - currEstimate)
      currEstimate = newEstimate
    
    if iterationCount >= maxIterations and currError > tolerance:
      print("Reached max iteration count without converging.")
      return

    result, currCoefficients = Horner(currCoefficients, currEstimate)
    roots[i] = currEstimate
    iterationCounts[i] = iterationCount

  return [roots, iterationCounts]

##### __Composite Midpoint Quadrature Formula__: uses midpoints of subintervals for integration

In [ ]:
def MidpointC(leftBound, rightBound, intervalCount, currFunction):
  intervalSize = (rightBound - leftBound) / intervalCount
  midpoints = np.linspace(leftBound, rightBound, intervalCount)
  functionMP = currFunction(midpoints) * np.ones(intervalCount)
  return intervalSize * sum(functionMP)

##### __Gauss LU Factorization__: decomposes a matrix into an upper(U) matrix and lower(L) matrix with diagonal elements equal to 1

In [ ]:
def GaussLU(mat):
  height = len(mat)
  width = len(mat[0])
  if height != width:
    print("Error: mat is not a square matrix!")
  else:
    for k in range(1, height):
      for i in range(k + 1, height + 1):
        mat[i][k] = mat[i][k] / mat[k][k]
        if mat[k][k] == 0:
          print("Error: Null diagonal element!")
        j = range(k + 1, height + 1)
        mat[i][j] = mat[i][j] - (mat[i][k] * mat[k][j])
  return mat

## Utility Functions

Function to store the system of equations:

In [ ]:
#Used for systems
def FunctionSystem(x):
  return np.array([(x[0]**2) + (x[1]**2) - 1,
                   np.sin(np.pi * x[0] / 2) + (x[1]**3)])

#Used for single functions
def CurrFunction(x):
  return x**2

## Test Ground

In [ ]:
mynum = np.array([2.0, -12.0])

#myAns = BroydenSys(FunctionSystem, mynum, 0.00001, 100)
#myAns = Aitken(CurrFunction, 1.44, 0.001, 100)
#myAns = Horner(mynum, 4.0)
#myAns = NewtonHorner(mynum, 5.8, 0.000000001, 100)
#myAns = MidpointC(0.0, 3.0, 500, CurrFunction)

myAns

9.0000009000001